# Нечеткие автоматы

In [ ]:
import numpy as np
import torch

In [ ]:
import sys
sys.path.insert(0, './python')

In [ ]:
from fuzzy_torch import logic
from fuzzy_torch.modules import ffsa, indicators

## Проверка переходов

In [ ]:
Logic = logic.Hamacher
fuzzy_fsa = ffsa.TimeIndependentFFSA(Logic)

In [ ]:
# Добавление состояний и переходов.
fuzzy_fsa.states = [0, 1]
fuzzy_fsa.transitions.append(ffsa.FuzzyTransition(0, 1, indicators.Sigmoid(1, 1.0, 0.0)))

In [ ]:
# Начальные активации.
activations = torch.tensor(np.array([[1.0, 0.0],
                                     [1.0, 0.0],
                                     [1.0, 0.0],
                                     [1.0, 0.0]]).astype(np.float32), requires_grad=True)

# Последовательности.
sequence = torch.tensor(np.array([[-1.0, -1.0, 0.0, 1.0, 1.0],
                                  [-10.0, -5.0, -1.0, 0.0, 10.0],
                                  [-10.0, -10.0, -10.0, -10.0, 0.0],
                                  [-0.5, -0.5, -0.5, -0.5, -0.5]]).astype(np.float32), requires_grad=True)

In [ ]:
Activations = [activations]
for step in range(sequence.size()[1]):
    Activations.append(fuzzy_fsa(sequence[:, step][:,None], Activations[-1]))

In [ ]:
torch.sum(Activations[-1][:,1]).backward()

In [ ]:
Activations

In [ ]:
sequence.grad

In [ ]:
print(fuzzy_fsa.transitions[0].condition.linear.weight.grad)
print(fuzzy_fsa.transitions[0].condition.linear.bias.grad)

## Последовательность с переключением

In [ ]:
class SwitchingRegressor(torch.nn.Module):
    def __init__(self, logic, ffsa):
        super().__init__()
        self.logic = logic
        self.ffsa = ffsa
        
    def forward(input, init_activations):
        steps = input.size()[1]
        
        activations = init_activations
        outputs = []
        for step in range(steps):
            # Срез входа по текущему шагу.
            input_on_current_step = input[:, step]
            
            # Новые активации (согласно нечеткому конечному автомату).
            activations = self.ffsa(input_on_current_step, activations)
            
            # Получение выходов регрессоров.
            output = [state[index](input_on_current_step) for state in self.ffsa.states]
            output = torch.stack(output, dim=1)
            outputs.append(activations * output)
            
        return torch.stack(outputs, dim=1)

In [ ]:
Logic = logic.Hamacher
regressor = SwitchingRegressor(Logic, ffsa.TimeIndependentFFSA(Logic))

In [ ]:
# Добавление состояний и переходов.
regressor.ffsa.states = [torch.nn.Linear(), torch.nn.Linear(), torch.nn.Linear()]
regressor.ffsa.transitions.append(ffsa.FuzzyTransition(0, 1, indicators.Sigmoid(1, 1.0, 0.0)))